In [1]:
DATA_NAME = 'pastis-full' 
TRANSFORM = 'fourier'
CHANNEL = 'gray'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_fourier_full_pastis.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10,11,...,24,25,26,27,28,29,30,31,32,33
2,10.13,-0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,-0.0,...,0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.0,0.0,-0.0
3,-0.00,7.83,0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,-0.0,...,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00,0.0,-0.0,-0.0
4,0.00,0.00,6.77,0.00,-0.00,-0.00,0.00,-0.00,0.00,0.0,...,0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.0,0.0,0.0
5,-0.00,0.00,0.00,5.42,0.00,-0.00,0.00,0.00,0.00,0.0,...,0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.0,-0.0,0.0
6,-0.00,0.00,-0.00,0.00,4.48,0.00,0.00,-0.00,-0.00,-0.0,...,-0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.0,0.0,0.0
7,0.00,0.00,-0.00,-0.00,0.00,3.58,0.00,-0.00,0.00,0.0,...,-0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.0,-0.0,0.0
8,0.00,-0.00,0.00,0.00,0.00,0.00,2.97,0.00,0.00,0.0,...,-0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,0.0,-0.0,-0.0
9,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,2.48,-0.00,0.0,...,0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.0,0.0,-0.0
10,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,2.05,0.0,...,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.0,0.0,0.0
11,-0.00,-0.00,0.00,0.00,-0.00,0.00,0.00,0.00,0.00,1.7,...,0.00,-0.00,0.00,-0.00,-0.00,0.00,-0.00,0.0,-0.0,-0.0


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10,11,...,24,25,26,27,28,29,30,31,32,33
2,1.00000,-0.00007,0.00002,-0.00003,-0.00002,0.00004,0.00003,-0.00004,0.00003,-0.00000,...,0.00004,0.00003,0.00004,-0.00005,-0.00000,-0.00000,0.00006,0.00003,0.00007,-0.00002
3,-0.00007,1.00000,0.00000,0.00002,0.00003,0.00005,-0.00002,-0.00003,0.00004,-0.00000,...,-0.00003,-0.00004,-0.00004,-0.00002,-0.00002,0.00000,0.00001,0.00003,-0.00002,-0.00006
4,0.00002,0.00000,1.00000,0.00007,-0.00003,-0.00001,0.00002,-0.00001,0.00002,0.00000,...,0.00001,-0.00001,-0.00001,-0.00003,0.00003,-0.00003,-0.00000,0.00005,0.00001,0.00005
5,-0.00003,0.00002,0.00007,1.00000,0.00004,-0.00002,0.00002,0.00005,0.00001,0.00000,...,0.00002,0.00003,-0.00001,-0.00003,-0.00003,0.00003,-0.00000,0.00004,-0.00001,0.00001
6,-0.00002,0.00003,-0.00003,0.00004,1.00000,0.00000,0.00003,-0.00001,-0.00002,-0.00000,...,-0.00000,0.00001,-0.00000,-0.00002,0.00002,-0.00001,-0.00000,0.00002,0.00002,0.00001
7,0.00004,0.00005,-0.00001,-0.00002,0.00000,1.00000,0.00002,-0.00000,0.00001,0.00000,...,-0.00006,-0.00001,-0.00000,-0.00001,0.00004,0.00002,-0.00007,-0.00005,-0.00003,0.00008
8,0.00003,-0.00002,0.00002,0.00002,0.00003,0.00002,1.00000,0.00007,0.00004,0.00000,...,-0.00005,0.00000,0.00002,0.00000,-0.00007,-0.00002,-0.00000,0.00001,-0.00004,-0.00003
9,-0.00004,-0.00003,-0.00001,0.00005,-0.00001,-0.00000,0.00007,1.00000,-0.00007,0.00001,...,0.00001,-0.00007,-0.00001,-0.00001,-0.00005,-0.00002,-0.00001,0.00001,0.00003,-0.00000
10,0.00003,0.00004,0.00002,0.00001,-0.00002,0.00001,0.00004,-0.00007,1.00000,0.00005,...,0.00001,-0.00001,0.00001,-0.00006,0.00002,-0.00001,0.00002,-0.00001,0.00000,0.00001
11,-0.00000,-0.00000,0.00000,0.00000,-0.00000,0.00000,0.00000,0.00001,0.00005,1.00000,...,0.00002,-0.00003,0.00001,-0.00001,-0.00003,0.00002,-0.00005,0.00001,-0.00003,-0.00002


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.0017375663927373304

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[3.22003481e+00 1.92025235e+00 1.43104259e+00 9.18208512e-01
 6.25616632e-01 3.99231376e-01 2.75025258e-01 1.90874484e-01
 1.29498960e-01 8.88015222e-02 5.95370104e-02 4.37553652e-02
 2.64157346e-02 1.89185853e-02 1.24820558e-02 8.16234771e-03
 5.21328054e-03 3.30403887e-03 2.04283256e-03 1.31195254e-03
 7.65478900e-04 4.38992439e-04 2.45754401e-04 1.36810229e-04
 6.84579340e-05 3.22234678e-05 1.46096519e-05 6.03526873e-06
 9.37839302e-07 2.05534276e-07 3.36237158e-08 6.34485218e-31]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.997117,-0.057964,-0.035624,-0.021951,-0.016006,-0.011665,-0.009267,-0.007551,-0.006062,-0.004971,...,-0.000257,-0.000194,-0.000137,-0.000100,-0.000066,-0.000044,-0.000016,-0.000008,-0.000003,-0.000002
1,0.051906,0.991192,-0.109093,-0.039638,-0.024964,-0.016803,-0.012944,-0.010263,-0.008107,-0.006589,...,-0.000343,-0.000259,-0.000185,-0.000127,-0.000087,-0.000057,-0.000022,-0.000011,-0.000005,-0.000003
2,0.038083,0.101619,0.989319,-0.079386,-0.040598,-0.024912,-0.018362,-0.014266,-0.011150,-0.008965,...,-0.000453,-0.000343,-0.000245,-0.000170,-0.000113,-0.000078,-0.000030,-0.000014,-0.000006,-0.000003
3,0.024039,0.041902,0.068129,0.990397,-0.093935,-0.040281,-0.026631,-0.019486,-0.014791,-0.011643,...,-0.000563,-0.000423,-0.000305,-0.000213,-0.000146,-0.000094,-0.000038,-0.000018,-0.000008,-0.000004
4,0.018853,0.028868,0.039626,0.083456,0.989603,-0.084283,-0.043580,-0.029016,-0.020821,-0.015878,...,-0.000731,-0.000549,-0.000393,-0.000274,-0.000184,-0.000124,-0.000049,-0.000024,-0.000010,-0.000005
5,0.013780,0.019630,0.024901,0.039197,0.071465,0.988352,-0.103706,-0.049114,-0.030698,-0.021951,...,-0.000922,-0.000687,-0.000490,-0.000340,-0.000227,-0.000153,-0.000064,-0.000032,-0.000014,-0.000005
6,0.011538,0.015963,0.019471,0.027873,0.041578,0.089482,0.985521,-0.112093,-0.050471,-0.032244,...,-0.001168,-0.000869,-0.000618,-0.000429,-0.000299,-0.000197,-0.000078,-0.000038,-0.000018,-0.000008
7,0.009933,0.013413,0.016052,0.021773,0.029939,0.049088,0.095652,0.984223,-0.111960,-0.052696,...,-0.001470,-0.001117,-0.000789,-0.000545,-0.000377,-0.000251,-0.000099,-0.000048,-0.000021,-0.000010
8,0.008254,0.010985,0.013017,0.017178,0.022475,0.032723,0.049412,0.093672,0.983173,-0.119334,...,-0.001845,-0.001388,-0.000987,-0.000691,-0.000464,-0.000313,-0.000122,-0.000061,-0.000027,-0.000012
9,0.007100,0.009370,0.010999,0.014240,0.018110,0.024899,0.034150,0.051072,0.099970,0.981930,...,-0.002339,-0.001767,-0.001252,-0.000870,-0.000595,-0.000394,-0.000159,-0.000077,-0.000035,-0.000016


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0028827294355249222,
 0.008808452257685229,
 0.010680871906150635,
 0.009602531301598938,
 0.010397488398045218,
 0.011648417175994852,
 0.014479296867976554,
 0.01577687903496816,
 0.016827395168982795,
 0.018070476508150524,
 0.02323021800938152,
 0.023743807755449486,
 0.022687131123236393,
 0.027544257090958424,
 0.025893836230398537,
 0.02668055551078019,
 0.027850095207383685,
 0.029566015696628423,
 0.033930488249139246,
 0.03648567758307775,
 0.03468012402047638,
 0.03724732531441666,
 0.042028266378601686,
 0.04493791110461276,
 0.044694416616948174,
 0.04989974829076882,
 0.06004020151358813,
 0.04948737404672432,
 0.034706913773582815,
 0.058550247205022554,
 0.12804459410077762,
 0.11024054114144388]